In [11]:
# %load src/header.py
'''LOAD'''
# %load /Users/rokushou/Desktop/header.py
'''GENERAL'''
import os
#import time
#import timeit
#import warnings
#warnings.simplefilter('ignore')
import random
import collections
#import secrets
import math
import itertools
'''DATA'''
import numpy as np
import pandas as pd
import databricks.koalas as ks
'''MONGO'''
#from pymongo import MongoClient
'''PLOT'''
import matplotlib.pyplot as plt
plt.style.use('seaborn-pastel')
font = {'size':16}
import seaborn as sns
'''SCIPY/STATS'''
#import scipy.stats as scs
from scipy.optimize import nnls
#import statsmodels.api as sm
'''SCIKIT LEARN'''
#from sklearn.datasets import make_classification, load_iris, load_boston, load_digits
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, classification_report, make_scorer
#from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
#from sklearn.linear_model import LinearRegression, LogisticRegression
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostRegressor, AdaBoostClassifier
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
'''TENSORFLOW'''
#import tensorflow as tf
'''NLTK'''
#import nltk
#import unicodedata
#import string
#from nltk.corpus import stopwords
#from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer
#from nltk.stem.wordnet import WordNetLemmatizer
'''NETWORK'''
import networkx as nx
import nxpd
import community as comm
'''FUNCTIONS'''
nrange = lambda x : itertools.repeat(None, x)
def argpsort(a, k):
    return np.argpartition(a, range(k))[:k]
def arghsort(a, k):
    b = np.argpartition(a, k)[:k]
    return b[np.argsort(a[b])]
'''PALETTE'''
t = 'setsuna'
qan = {'setsuna':'quanta'}
twi = '#DCB8E7' #Pale, light grayish mulberry
twi_blu = '#273873' #Dark sapphire blue
twi_pur = '#662D8A' #Moderate purple
twi_pnk = '#ED438D' #Brilliant raspberry
'''RETURN'''
!fortune | cowsay -f dragon

 _________________________________________ 
/ If I could stick my pen in my heart, I  \
| would spill it all over the stage.      |
| Would it satisfy ya, would it slide on  |
| by ya, Would you think the boy was      |
| strange? Ain't he strange? ... If I     |
| could stick a knife in my heart,        |
| Suicide right on the stage, Would it be |
| enough for your teenage lust, Would it  |
| help to ease the pain? Ease your brain? |
|                                         |
| -- Rolling Stones, "It's Only Rock'N    |
\ Roll"                                   /
 ----------------------------------------- 
      \                    / \  //\
       \    |\___/|      /   \//  \\
            /0  0  \__  /    //  | \ \    
           /     /  \/_/    //   |  \  \  
           @_^_@'/   \/_   //    |   \   \ 
           //_^_/     \/_ //     |    \    \
        ( //) |        \///      |     \     \
      ( / /) _|_ /   )  //       |      \     _\
    ( // /) '/,_ _ _/  ( ; -.    |    _ 

# Initial Cleaning

I have a directory of csvs from stat.ink which contains player submitted match data. I want to process them into one cleaned database. Leveraging the power of Koalas, I can use a Spark backend with a Pandas-like interface. I will first clean one csv to devolop a pipeline.

In [187]:
# Gather list of raw data filenames
raw_lst = !ls data/raw
# Load one to EDA
df = pd.read_csv('data/raw/' + raw_lst[0])
df.head(3)

,# period,game-ver,lobby-mode,lobby,mode,stage,time,win,knockout,A1-weapon,...,B3-level,B4-weapon,B4-kill-assist,B4-kill,B4-assist,B4-death,B4-special,B4-inked,B4-rank,B4-level
0,2017-08-07T04:00:00+00:00,1.1.2,gachi,squad_2,hoko,ama,300,alpha,False,splatscope,...,29,hokusai,7,6,1,5,4,1353,b-,33
1,2017-08-08T02:00:00+00:00,1.1.2,gachi,standard,yagura,hokke,221,alpha,True,splatscope,...,22,96gal,6,5,1,8,1,736,b,27
2,2017-08-08T02:00:00+00:00,1.1.2,gachi,standard,yagura,chozame,312,bravo,False,splatscope,...,19,nzap85,10,5,5,9,3,696,b+,24


### Preliminary Feature Selection

Since this data is from match results and I want to create a predictive model for win rate, there are many features that cannot be used as they are only knowable at the end of a match:

- period: when the match was played
    - irrelevant because this is not a timeseries model
    - version number better covers patch changes
- time: how long the match took
- knockout: if the match was won through the objective as opposed to a time-out
- player kill/assist/death/special/inked: player statistics calculated at the end of each match

In [188]:
def splatoon_drop(df):
    """ 
    Drops unnessecary features in stat.ink dataframes
  
    Parameters: 
    df (DataFrame): Pandas dataframe on which to drop values
  
    Returns: 
    None: function is inplace 
  
    """
    # initialize list with first features
    drop_lst = ['# period','time', 'knockout']
    # concatanate player statistics features for each player
    for player in ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4']:
        drop_lst += [player+'-kill-assist', player+'-kill', player+'-assist',
                     player+'-death', player+'-special', player+'-inked']
    # drop features in list
    df.drop(drop_lst, axis=1, inplace=True)
    return None

In [189]:
splatoon_drop(df)
df.head(3)

,game-ver,lobby-mode,lobby,mode,stage,win,A1-weapon,A1-rank,A1-level,A2-weapon,...,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
0,1.1.2,gachi,squad_2,hoko,ama,alpha,splatscope,a-,27,sshooter_collabo,...,22,sshooter,b-,29,hokusai,a,29,hokusai,b-,33
1,1.1.2,gachi,standard,yagura,hokke,alpha,splatscope,b,28,nzap85,...,12,dynamo,b+,23,hissen,b+,22,96gal,b,27
2,1.1.2,gachi,standard,yagura,chozame,bravo,splatscope,b,28,wakaba,...,24,promodeler_mg,b-,15,splatroller,b,19,nzap85,b+,24


### Exploding the rows

Since I am interested in making predictions based off data from one player, I will explode each match data into 8 separate rows, one for every player. The function will drop data from A1 by default as A1 is the player who submitted the data, which can lead to bias. I have more than enough data to drop it.

In [173]:
def splatoon_explode(df, drop=True):
    """ 
    Explodes rows of stat.ink dataframes into one row for each player.
    This function is intended to be run after splatoon_drop and
    is hardcoded to accept its output features.
  
    Parameters: 
    df (DataFrame): Pandas dataframe on which to explode
  
    Returns: 
    koalas: Exploded dataframe stored in a koalas database
  
    """
    # get feature names
    features = df.columns.tolist()
    # features for all players
    shared = features[:5]
    # features for specific player
    a1 = features[6:9]
    a2 = features[9:12]
    a3 = features[12:15]
    a4 = features[15:18]
    b1 = features[18:21]
    b2 = features[21:24]
    b3 = features[24:27]
    b4 = features[27:]
    # group players by team
    if drop:
        a_team = [a2, a3, a4]
    else:
        a_team = [a1, a2, a3, a4]
    b_team = [b1, b2, b3, b4]

    # initialize temporaty storage list
    tmp = []
    # iterate through rows
    for index, row in df.iterrows():
        # get features common to both teams
        both = [index] + row[shared].tolist()
        # append features for each a team player
        for player in a_team:
            tmp.append(both + row[player].tolist() + [row.win == 'alpha'])
        # append features for each b team player
        for player in b_team:
            tmp.append(both + row[player].tolist() + [row.win == 'bravo'])

    #create new column names, including the index of the match the data was from
    new_cols = ['match'] + general + ['weapon', 'rank', 'level', 'win']
    # return new koalas database
    return ks.DataFrame(tmp, columns=new_cols)

In [191]:
df2 = splatoon_explode(df)
df2.head(3)

,match,game-ver,lobby-mode,lobby,mode,stage,weapon,rank,level,win
0,0,1.1.2,gachi,squad_2,hoko,ama,sshooter_collabo,b+,25,True
1,0,1.1.2,gachi,squad_2,hoko,ama,sshooter,b-,30,True
2,0,1.1.2,gachi,squad_2,hoko,ama,hokusai,b+,23,True


In [196]:
files = raw_lst[:2]
path = 'data/raw/'

df = pd.read_csv(path+files[0])
for file in files[1:]:
    df = pd.concat([df, pd.read_csv(path+file)])

In [197]:
df

,# period,game-ver,lobby-mode,lobby,mode,stage,time,win,knockout,A1-weapon,...,B3-level,B4-weapon,B4-kill-assist,B4-kill,B4-assist,B4-death,B4-special,B4-inked,B4-rank,B4-level
0,2017-08-07T04:00:00+00:00,1.1.2,gachi,squad_2,hoko,ama,300,alpha,False,splatscope,...,29,hokusai,7,6,1,5,4,1353,b-,33
1,2017-08-08T02:00:00+00:00,1.1.2,gachi,standard,yagura,hokke,221,alpha,True,splatscope,...,22,96gal,6,5,1,8,1,736,b,27
2,2017-08-08T02:00:00+00:00,1.1.2,gachi,standard,yagura,chozame,312,bravo,False,splatscope,...,19,nzap85,10,5,5,9,3,696,b+,24
3,2017-08-08T02:00:00+00:00,1.1.2,gachi,standard,yagura,chozame,300,alpha,False,splatscope,...,20,hokusai,20,12,8,10,2,702,b,28
4,2017-08-09T00:00:00+00:00,1.1.2,gachi,standard,hoko,chozame,304,alpha,False,sputtery,...,33,sputtery,6,3,3,9,5,1252,s+,33
5,2017-08-09T00:00:00+00:00,1.1.2,gachi,standard,hoko,hokke,304,alpha,False,sputtery,...,38,dualsweeper,8,7,1,7,4,956,s+,32
6,2017-08-09T00:00:00+00:00,1.1.2,gachi,standard,hoko,chozame,300,alpha,False,sputtery,...,28,sputtery,5,1,4,9,1,671,s+,39
7,2017-08-09T00:00:00+00:00,1.1.2,gachi,standard,hoko,hokke,67,bravo,False,sputtery,...,24,rapid,4,4,0,1,1,419,s+,40
8,2017-08-09T00:00:00+00:00,1.1.2,gachi,standard,hoko,chozame,77,bravo,False,sputtery,...,28,NaN,2,2,0,2,0,290,s+,31
9,2017-08-09T02:00:00+00:00,1.1.2,gachi,standard,area,kombu,239,bravo,False,sputtery,...,38,hissen,9,6,3,5,4,956,s+,31
